In [1]:
import pandas as pd
import random
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
INPUT_FILE = '../toxicity_ratings - twitter.csv'

In [4]:
df = pd.read_csv(INPUT_FILE)
len(df)

/Users/shagunjhaver/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6,7,8,9,10,14,15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


72947

In [5]:
df = df.dropna()
len(df)

70208

In [6]:
toxic_fields = [
    'toxic_score',
    'toxic_score.1',   
    'toxic_score.2',
    'toxic_score.3',    
    'toxic_score.4',    
]

In [7]:
def get_avg_toxicity(row):
    result = sum(row[field] for field in toxic_fields)/20.0
    for field in toxic_fields:
        diff = abs(row[field]/4.0 - result)
        if (diff > 0.2):
            return None
    return result

In [8]:
df['avg_toxic_score'] = df.apply (lambda row: get_avg_toxicity(row), axis=1)

In [9]:
df = df.dropna()
len(df)

27994

In [10]:
def get_bucket(row, field):
    if (row[field] < 0.6):
        return 0.0
    elif (row[field] < 0.7):
        return 0.6
    elif (row[field] < 0.8):
        return 0.7
    elif (row[field] < 0.9):
        return 0.8
    else:
        return 0.9

In [11]:
df['avg_toxic_window'] = df.apply(lambda row: get_bucket(row, 'avg_toxic_score'), axis=1)

In [12]:
myrange = [0.0, 0.6, 0.7, 0.8, 0.9]
sampled_dfs = []
for ow in myrange:
    g = df[df['avg_toxic_window'] == ow]
    print (ow, len(g))
    sampled_dfs.append(g.sample(4))

0.0 27710
0.6 111
0.7 30
0.8 61
0.9 82


In [13]:
final_df = pd.concat(sampled_dfs)

In [14]:
def get_random_date():
    random_number_of_days = random.randrange(40)
    random_date = datetime.date(2021, 10, 1) + datetime.timedelta(days=random_number_of_days)
    return random_date

In [15]:
def assignPeripherals(final_df):
    final_df = final_df[['comment', 'avg_toxic_score', 'comment_ID']]
    
    # Set username
    usernameDf = pd.read_csv('usernames.csv')
    usernames = set(list(usernameDf['username']))
    final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
    
    # Set full name
    nameDf = pd.read_csv('random_names.csv')
    names = set(list(nameDf['name']))
    final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)    
    
    # Set integer fields
    final_df['num_likes'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_comments'] = np.random.randint(1, 100, final_df.shape[0])    
    final_df['num_retweets'] = np.random.randint(1, 100, final_df.shape[0])        
    
    # Set publication date
    final_df['pub_date'] = df.apply(lambda row: get_random_date(), axis=1)
    
    return final_df

In [16]:
final_df = assignPeripherals(final_df)
final_df.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
5261,"We also know that Mueller, per his report cannot connect the IRA aka the trolls to the Russian government. We know no one was indicted for conspiracy between Trump, Trump campaign and Russia",0.00,1548_14,thefrizzyforgiveness,Sydney Adams,20,55,75,2021-10-11
2822,Quit tweeting at the wee hours of night,0.05,81_21,kosheropposition_47,Lucy Holmes,48,27,26,2021-11-02
37407,pround of you (·•︠‿•︡ )♡ #부산의아들_강다니엘,0.05,2547_7,PhysicalTendon,Julia Montgomery,81,96,28,2021-10-12
58647,Kinda pissed off with tealive usj 9,0.05,5659_10,humongous_bingo,Arnold Harper,45,35,12,2021-11-09
45891,Wtf does this even mean you crack head???,0.65,2475_20,exciting_self-control,Kate Farrell,86,98,99,2021-10-28


In [17]:
final_df.to_csv('main_comments.csv', index=False)

In [18]:
nontoxic_df = df.query('avg_toxic_window == 0.0')

In [19]:
nontoxic_sample = nontoxic_df.sample(16)

In [20]:
nontoxic_sample = assignPeripherals(nontoxic_sample)
nontoxic_sample.head()

<ipython-input-15-61dd48437cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author'] = df.apply(lambda row: random.sample(usernames, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['author_name'] = df.apply(lambda row: random.sample(names, 1)[0], axis=1)
<ipython-input-15-61dd48437cb0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


,comment,avg_toxic_score,comment_ID,author,author_name,num_likes,num_comments,num_retweets,pub_date
47014,Yoooo she’s dope!! Need art done hit her up,0.00,4849_11,thecrispmetaphysics,Madaline Douglas,48,90,32,2021-10-25
13780,You not ever gon catch me doing too much on social media.,0.05,1456_17,PM_ME_YOUR_MAYORS,Emily Roberts,69,55,47,2021-11-09
3071,You said to be responsible.. so wouldn't the responsible choice be to tell Mom and Dad? Or maybe that would be the stupid choice..,0.00,1008_11,high_and_mighty,Connie Owens,65,47,73,2021-10-30
22024,I cheer for the #SuperEagles in the #TotalAFCON2019 against South Africa 🇿🇦! Go Nigeria! #NGARSA #FootballTogether,0.00,840_23,feeble_cynicism,Honey Crawford,95,62,49,2021-11-03
34297,I did the creamer last week and they brought it back again this week. Limited number of uses for the the Ritz I think,0.00,2695_13,high_and_mighty,Tara Higgins,1,76,44,2021-10-23


In [21]:
nontoxic_sample.to_csv('nontoxic_comments.csv', index=False)